In [230]:
%load_ext autoreload
%autoreload 2
# So code is automatically reloaded when saved in different modules

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
  
import os
import pickle
import numpy as np
import tensorflow as tf
from scipy import stats
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import sklearn.metrics
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
sys.path.append('src/taggerSystem/')
from my_util import print_sentence, write_conll, read_conll
from my_data_util import load_and_preprocess_data, load_embeddings, ModelHelper
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

## File Paths


In [232]:
data_train = "data/icd9NotesDataTable_train.csv"
data_valid = "data/icd9NotesDataTable_valid.csv"
# data_train = "data/smallIcd9NotesDataTable_test.csv"
# data_valid = "data/smallIcd9NotesDataTable_valid.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
output_path = 'results/{}/{:%Y%m%d_%H%M%S}/".format(self.cell, datetime.now())'
log_output = output_path + "log"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50

## Extract Data and Word Embeddings

In [233]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid)
# print('dev raw')
# print(dev_raw)
# print('dev')
# print(dev)
embeddings = load_embeddings(vocab, wordVecs, helper)
helper.save(output_path)# token2id and max length saved to output_path
handler = logging.FileHandler(log_output)
handler.setLevel(logging.DEBUG)
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
logging.getLogger().addHandler(handler)

INFO:Loading training data...
INFO:Done. Read 39544 notes
INFO:Loading dev data...
INFO:Done. Read 13182 notes
INFO:Total read time 16.562712
INFO:Built dictionary for 10007 features.
INFO:There are a total of 20 ICD codes
INFO:Initialized embeddings.


## Hyperparameters

In [241]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 256
total_batches = (xTrain.shape[0]//batch_size)

n_input = 1
n_steps = 10
n_hidden = 30
n_classes = helper.n_labels
n_features = 1

alpha = 0.5

In [242]:
total_batches

154

In [243]:
helper.max_length

500

In [244]:
xTrain.shape

(39544, 500)

In [245]:
yTrain.shape

(39544, 20)

## Placeholders

In [246]:
# x = tf.placeholder(tf.int32, shape= (None, helper.max_length, n_features))
x = tf.placeholder(tf.int32, shape= (None, helper.max_length))
yTruth = tf.placeholder(tf.int32, shape = (None, helper.n_labels))
y_steps = tf.placeholder(tf.int32, shape = (None, helper.n_labels))# not sure what this is

## Helper Functions

In [247]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,state_is_tuple = True)
    print(type(cell))
    print(cell)
    print(type(x))
    print(x.get_shape())
    print('cell output size')
    print(cell.output_size)
    print('cell state size')
    print(cell.state_size)
#     1/0
#     print('cell size')
#     print(cell.get_shape())
#     1/0
#     multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2)

    output, state = tf.nn.dynamic_rnn(cell = cell, inputs = x, dtype = tf.float32)
    print('output shape')
    print(output.get_shape())
#     print(tf.transpose(output,[1,0,2]).get_shape())
#     temp = tf.gather(tf.transpose(output,[1,0,2]), helper.max_length-1) 
#     print('temp shape')
#     print(temp.get_shape())
#     1/0
#     output_flattened = tf.reshape(output, [-1, n_hidden])
    output_flattened = tf.gather(tf.transpose(output,[1,0,2]), helper.max_length - 1)
    print('output flattened shape')
    print(output_flattened.get_shape())
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    # TODO here is where you're gonna have to change some stuff. You're
    # running out of memory with this run. It'd be best to not multiply
    # and get all predictions since we really only need last one

#     output_all = tf.nn.sigmoid(output_logits)
    print('output wx + b')
    print(output_logits.get_shape())
#     1/0
#     output_reshaped = tf.reshape(output_logits,[-1,n_steps,n_classes])
#     output_reshaped = tf.reshape(output_logits,[-1,helper.max_length,n_classes])
#     print('reshaped shape')
#     print(output_reshaped.get_shape())
#     output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), helper.max_length - 1)
    output_last = output_logits
    print('output last shape')
    print(output_last.get_shape())
    #Here is where you wanna get the last one. 
    #output = tf.transpose(output, [1, 0, 2])
    #last = tf.gather(output, int(output.get_shape()[0]) - 1)
    #output_last = tf.nn.sigmoid(tf.matmul(last, weight) + bias)
    return output_last

In [249]:
# weight = weight_variable([n_hidden,n_classes])
with tf.variable_scope('RNN_OutsideCell', reuse = False) as scope:
    U = tf.get_variable(name = 'U', shape = (n_hidden, n_classes), 
                    initializer = tf.contrib.layers.xavier_initializer())
    bias = tf.get_variable(name = 'bias', shape = [n_classes], 
                           initializer = tf.constant_initializer(0))
    # bias = bias_variable([n_classes])
    pretrainedEmbeddings = tf.Variable(embeddings)
    wordEmbeddings = tf.nn.embedding_lookup(params = pretrainedEmbeddings, ids = x)
    print(wordEmbeddings.get_shape())
#     wordEmbeddings = tf.reshape(wordEmbeddings, 
#                                 shape = tf.stack([-1, wordEmbeddings.get_shape()[1], wordEmbeddings.get_shape()[2]*wordEmbeddings.get_shape()[3]])) 
    print('shape of embeddings')
    print(wordEmbeddings.get_shape())
    print('U shape')
    print(U.get_shape())
    print('bias shape')
    print(bias.get_shape())
    y_last = LSTM(wordEmbeddings,U,bias)# TODO is y_last the correct thing to return?
# adding loss function

(?, 500, 50)
shape of embeddings
(?, 500, 50)
U shape
(30, 20)
bias shape
(20,)
<class 'tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell'>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 500, 50)
cell output size
30
cell state size
LSTMStateTuple(c=30, h=30)
output shape
(?, 500, 30)
output flattened shape
(?, 30)
output wx + b
(?, 20)
output last shape
(?, 20)


## Loss Function

In [250]:
batchError = tf.nn.sigmoid_cross_entropy_with_logits(logits = y_last, 
                                                 labels = tf.cast(yTruth, tf.float32))
# print('yolo2')
loss_function = tf.reduce_mean(batchError)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

/home/sandeepayyar1/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [251]:
# The paper does something weird. I'd suggest doing something closer to what we do in homework 3

## Training and Testing

In [252]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
# hacky work around for issue where -1 is padding but now maps to UUNNNKKK vector.

In [ ]:
with open('runOutput.txt', 'w') as f:
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        for epoch in range(training_epochs):
            totalError = 0.0
            f.write("""***************************
***************************
Running on epoch %d
***************************
***************************
***************************\n""" %(epoch))
            print('***************************')
            print('***************************')
            print('Running on epoch %d'% (epoch))
            print('***************************')
            print('***************************')
            start = time.time()
            for b in range(total_batches):
        #             print(b)
        #             if(b%25 == 0):
        #                 print('running iteration %d'% (b))
                offset = (b * batch_size) % (yTrain.shape[0] - batch_size)
                batch_x = xTrain[offset:(offset + batch_size), :]
        #             print('batch x size')
        #             print(batch_x.shape)
                batch_y = yTrain[offset:(offset + batch_size), :]
        #             print('y shape')
        #             print(batch_y.shape)
        #             print(psutil.virtual_memory())
        #             1/0
        #             batch_y_steps = np.tile(batch_y,((xTrain.shape[1]),1))
        #             1/0
                _, c = session.run([train_op, loss_function],feed_dict={x: batch_x, yTruth : batch_y})
                if(b%25 == 0):
                    f.write('running iteration %d with error %3f \n'% (b, c))
                    print('running iteration %d with error %3f'% (b, c))
        #             1/0
                totalError = totalError + c
            pred_y = session.run(y_last,feed_dict={x:xDev})
        #         pred_y = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y)
        #         print("ROC AUC Score: ",roc_auc_score(yDev,pred_y))
            print('average Error %f'% (totalError/total_batches))
            print('Total run time was %3f'% (time.time() - start))
            f.write('average Error %f \n'%(totalError/total_batches)) 
            f.write('Total run time was %3f \n'% (time.time() - start))

***************************
***************************
Running on epoch 0
***************************
***************************
running iteration 0 with error 0.708766
running iteration 25 with error 0.592134
running iteration 50 with error 0.498013
running iteration 75 with error 0.496222
running iteration 100 with error 0.477900
running iteration 125 with error 0.467245
running iteration 150 with error 0.462870
average Error 0.519059
Total run time was 43.486848
***************************
***************************
Running on epoch 1
***************************
***************************
running iteration 0 with error 0.480418
running iteration 25 with error 0.470096
running iteration 50 with error 0.462857
running iteration 75 with error 0.476209
running iteration 100 with error 0.452238
running iteration 125 with error 0.445673
running iteration 150 with error 0.441331
average Error 0.462731
Total run time was 42.929129
***************************
***************************


In [264]:
pred_y.shape

(13182, 20)

In [260]:
sess = tf.InteractiveSession()
np.sum(tf.argmax(pred_y, 1).eval())

13182

In [225]:
print("f1_score %f"% sklearn.metrics.f1_score(yDev, pred_y))

ValueError: Can't handle mix of multilabel-indicator and continuous-multioutput

In [239]:
temp = tf.argmax(pred_y, 1)

In [240]:
sess = tf.InteractiveSession()
temp.eval()

array([735, 735, 735, ..., 735, 735, 735])

In [215]:
pred_y.shape

(13182, 1851)

In [138]:
13182000/256

51492.1875

In [139]:
batch_x.shape

(256, 1000)

In [195]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [197]:
sess

In [185]:
print(b)

153


In [187]:
totalError/154

0.22618506132782279

In [192]:
6591000/(500)

13182.0

In [211]:
helper.icdDict

{'': 0,
 '624': 1,
 'V552': 2,
 'V1062': 6,
 '323': 7,
 '962': 8,
 '437': 9,
 '288': 10,
 'E8193': 11,
 'V2501': 12,
 '471': 13,
 '333': 14,
 'V1741': 15,
 'E8543': 16,
 '176': 17,
 '651': 18,
 'V0991': 19,
 'E8405': 20,
 '273': 21,
 '226': 22,
 'E9504': 1552,
 '183': 24,
 '823': 937,
 '611': 26,
 'V4976': 624,
 'E8210': 28,
 'E9220': 29,
 '718': 30,
 'V449': 31,
 'V561': 1616,
 '299': 32,
 'E9888': 33,
 'E8702': 34,
 '343': 627,
 'V0950': 939,
 '133': 37,
 'V145': 38,
 'V180': 39,
 'E8840': 1811,
 'V023': 40,
 '449': 41,
 'V5481': 42,
 '218': 43,
 'E8853': 267,
 '97': 45,
 '707': 46,
 '345': 47,
 '879': 48,
 'V4281': 49,
 'E8314': 50,
 '756': 51,
 'E8609': 52,
 'E9293': 53,
 'V671': 54,
 'E8584': 55,
 '161': 56,
 '999': 57,
 '180': 58,
 '972': 1384,
 'V643': 59,
 'E9294': 60,
 '7': 943,
 'V4986': 62,
 '698': 63,
 'E8497': 64,
 '875': 1764,
 'E0039': 65,
 'E9346': 66,
 'V1261': 67,
 'E9325': 68,
 'V440': 69,
 '438': 70,
 'E8624': 71,
 '377': 72,
 'V1505': 73,
 'E9331': 74,
 '659': 75,
